In [1]:
import requests
import re
from bs4 import BeautifulSoup
import csv

In [2]:
# Gerekli fonksiyonlari burada tanimladik

# For writing only one data(must be a tuple) at a time 
def writeRowToCsv(fileName, dataOnTuple):
  f = open(fileName, "a", newline="")
  writer = csv.writer(f)
  writer.writerow(dataOnTuple)
  f.close

# For writing datas(must be a tuple)
def writeRowsToCsv(fileName, datasOnTuple):
  with open(fileName, 'a', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(datasOnTuple)

# para stringini temizlemek icin fonksiyon
def clearPrice(price):
  p = price.replace(' ', '')
  p = p.replace('TL', '')
  p = p.replace('.', '')
  return p



In [3]:
#  Diğer traktör vb araç tiplerini almıyoruz ve her sayfada 50 araç göstersin istiyoruz. 02.12.2022 tarihinde 71.611 araç var. 
url = 'https://www.arabam.com/ikinci-el/sahibinden?take=50&page='

# Tarayicimizin user agent ini veriyoruz
headers = {"User-Agent" : "Mozilla/5.0 (X11; Linux x86_64; rv:107.0) Gecko/20100101 Firefox/107.0"}


In [14]:
# 50 sayfa linkleri
sayfa_linkleri = []
for i in range(1,50):
  sayfa_linkleri.append(url + str(i))

In [15]:
# ilan basligi-url lerini once list'e cekiyoruz
araba_linkleri = []

# her sayfada dolasip araba ilanlarinin linklerini topluyoruz
for sayfa in sayfa_linkleri:
  r = requests.get(url, headers=headers)
  # http 200 kod numarasi dondu ise islemleri baslatiyoruz
  if(r.status_code == 200):
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    table_rows = soup.find_all('tr', class_='listing-list-item pr should-hover bg-white')
    #print(table_rows[0])
    for row in table_rows:
      a = row.td.div.a
      link = "https://www.arabam.com" + a.get('href')
      araba_linkleri.append(link)
      writeRowToCsv("sayfa_linkleri_galeriden.csv", (link,))
      #print(link)

In [16]:
# sonra ise sayfada bulunan araba liklerine gidip her arabanin detayini cekiyoruz ve csv dosyamiza ekliyoruz
for link in araba_linkleri:
  r_cardata = requests.get(link, headers=headers)
  if(r_cardata.status_code == 200):
    html = r_cardata.text
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('div', class_='banner-column-detail bcd-mid-extended p10 bg-white')
    # preparing data
    # ilanNo, adres, ilanNo, ilanTarihi, marka, seri, model, 
    # Genel Bakis
    # yil, kilometre, vitesTipi, yakitTipi, kasaTipi, aracTuru, renk, plakaUyrugu, garantiDurumu, takasaUygun, aracinIlkSahibi, kimden, yillikMTV
    # cekis, silindirSayisi, tork, motorHacmi, motorGucu, hizlanma, maxHiz
    # ortYakit, sehirIciYakir, sehirDisiYakit, yakitDeposu
    # uzunluk, genislik, yukseklik, agirlik, bosAgirligi, koltukSayisi, bagajHacmi, onLastik, aksAraligi
    # boyaDegisen
    all_data = []
    all_data.append(link)
    price = table.div.div.span.get_text().strip()
    all_data.append(price.strip())
    all_data.append(table.p.get_text().strip())

    # finding all unordered lists
    all_spans = table.find_all('span', class_='bli-particle')
    i = 0
    while i in range(25):
      if (i%2 == 1):
        all_data.append(all_spans[i].get_text().strip())
      i += 1
    # writing header of the csv file
    #header = ("price", "location", "advert_no", "start_date", "brand", "series", "model", "year", "mileage", "vehicleTransmission", "fuelType", "bodyType", "engineDisplacement", "enginePower", "driveUnit", "fuelTank", "swappableness", "fromWhom")
    #writeRowToCsv("arabalar_data.csv", header)
    # all_data List is (price, location, advert_no, start_date, brand, series, model, year, mileage, vehicleTransmission, fuelType, bodyType, engineDisplacement, enginePower, driveUnit, fuelTank,  swappableness, fromWhom)
    # writing data to csv
    #link,fiyat,adres,ilan_no,ilan_tarihi,marka,seri,model,yil,kilometre,vites_tipi,yakit_tipi,kasa_tipi,motor_hacmi,motor_gucu
    writeRowToCsv("arabalar_data.csv", all_data)

ConnectionError: ('Connection aborted.', OSError(113, 'No route to host'))